In [47]:
import openai
import matplotlib.pyplot as plt
import pandas as pd
import requests
from PIL import Image # for converting the webp image to jpeg
import time, glob

In [ ]:
models = openai.Model.list()
models

In [ ]:
image_response = openai.Image.create(
        prompt="Create a vibrant and impressionistic scene titled 'The Flowering Orchard,' inspired by Van Gogh's style. The composition should feature a colorful orchard in full bloom, showcasing various flowering trees, particularly plum trees, with abundant pink and white blossoms. Include a picturesque view of Arles in the background, capturing the essence of a serene spring day. The sky should be a brilliant blue, and the foreground should be filled with lush greenery and the playful colors of flowers, creating a lively and inviting atmosphere. The overall mood should evoke the beauty and tranquility of nature in springtime.",
        n=1,
        size="1024x1024")
    
image_url = image_response['data'][0]['url']
print("Generated image URL: ", image_url)

response = requests.get(image_url)

if response.status_code == 200:
    with open("generated_image.png", 'wb') as file:
        file.write(response.content)
    print("Image successfully saved as 'generated_image.png'")
else:
    print("Failed to retrieve the image")
    

In [15]:
path_gen = "../data/artwork_gen_dalle"
path_orig = "../data/original_artworks_named"

df = pd.read_csv("../data/artworks_data_with_prompts_simple_generated_images.csv")

# how many rows in the dataframe has generated_artwork_name empty
len(df) - df['generated_artwork_name'].isnull().sum()

In [ ]:
df.at[100, 'generated_artwork_name']

In [ ]:
# sort files in path_gen by creation date
path_gen = "../data/artwork_gen_dalle"
path_orig = "../data/original_artworks_named"
files = glob.glob(path_gen + "/*")
len(files)


In [ ]:
count = 0
curr_t = time.time()
for index, row in df.iterrows():
 
    print("==========" , count , "==========")
    # if generated_artwork_name not empty, skip
    if pd.isnull(row['generated_artwork_name']):
        print("YAY I'm Already generated")
        count += 1
        continue

    prompt = row['prompt']
    # convert " to ' for the prompt
    prompt = prompt.replace('"', "'")
    art_name = row['artwork_name']

   
    img_name = f"{row[('artwork_name')]}_{row['artist_name']}.png"
    generated_img_path =  f"../data/artwork_gen_dalle/{img_name}"
    
    # search generated image in the path_gen
    found = False
    for file in files:
        if img_name in file:
            found = True
            break
            
    if not found:
        print("Image not found")
        continue
        
    img_full_path = row[('image_path_named')]
    original_img = Image.open(img_full_path)

    try :
        generated_img = Image.open(generated_img_path)
    except Exception as e:
        print("Error: ", e)
        df.at[index, 'generated_artwork_name'] = e
        count += 1
        continue


    # plot
    fig, axs = plt.subplots(1, 2, figsize=(12, 6))
    
    print(prompt)
    axs[0].imshow(original_img)
    axs[0].set_title("Original Image")
    axs[0].axis('off')

    axs[1].imshow(generated_img)
    axs[1].set_title("Generated Image")
    axs[1].axis('off')

    plt.show()
    
    print("generated_img_path", generated_img_path)
    df.at[index, 'generated_artwork_name'] = generated_img_path
    count += 1

In [ ]:
df.to_csv("../data/artworks_data_with_prompts_simple_generated_images.csv", index=False)

In [7]:
df = pd.read_csv("../data/artworks_data_with_prompts_simple_generated_images.csv")
df.head()

In [ ]:
# fill already generated images
# iterate over the rows of the dataframe
count = 0
curr_t = time.time()
for index, row in df.iterrows():
    # if count < 800:
    #     print("Skipping row ", count)
    #     count += 1
    #     continue
    # 
    # if count > 1010:
    #     break
    
    # do only five requests per minute
    # if count % 5 == 0:
    #     time_rem = 60 - (time.time() - curr_t)
    #     print("Sleeping for 60 seconds")
    #     time.sleep(0)
    #     curr_t = time.time()
        
    if not pd.isnull(row['generated_artwork_name']):
        print("YAY I'm Already generated")
        count += 1
        continue
    
    print("==========" , count , "==========")
    prompt = row['prompt']
    # convert " to ' for the prompt
    prompt = prompt.replace('"', "'")
    art_name = row['artwork_name']
    
    try:
        image_response = openai.Image.create(
            prompt=prompt,
            n=1,
            size="1024x1024")
        image_url = image_response['data'][0]['url']
        print("Generated image URL: ", image_url)
    except Exception as e:
        print("Error: ", e)
        row['generated_artwork_name'] = e
        if "Rate limit" in str(e):
            print("Sleeping for 60 seconds")
            time.sleep(60)
            try:
                image_response = openai.Image.create(
                    prompt=prompt,
                    n=1,
                    size="1024x1024")
                image_url = image_response['data'][0]['url']
                print("Generated image URL: ", image_url)
            except Exception as e:
                print("Error: ", e)
                count += 1
        
        image_response = "Illegal Prompt"
        continue

    
    if image_response == "Illegal Prompt":
        row['generated_artwork_name'] = "Illegal Prompt"
        count += 1
        continue
        
    # Download the image
    response = requests.get(image_url)
    
    img_generated_path = img_name = f"{row[('artwork_name')]}_{row['artist_name']}.png"
    generated_img_path =  f"../data/artwork_gen_dalle/{img_name}"
    # Save the image to a local file
    if response.status_code == 200:
        with open(generated_img_path, 'wb') as file:
            file.write(response.content)
            print("Image successfully saved")
    else:
        print("Failed to retrieve the image")
  
        
    img_full_path = row[('image_path_named')] 
   
    
    # plot
    original_img = Image.open(img_full_path)
    generated_img = Image.open(generated_img_path)

    fig, axs = plt.subplots(1, 2, figsize=(12, 6))
    
    print(prompt)
    axs[0].imshow(original_img)
    axs[0].set_title("Original Image")
    axs[0].axis('off')
    
    axs[1].imshow(generated_img)
    axs[1].set_title("Generated Image")
    axs[1].axis('off')

    plt.show()
    
    print("generated_img_path", generated_img_path)
    df.at[index, 'generated_artwork_name'] = generated_img_path
    count += 1

In [41]:
df.to_csv("../data/artworks_data_with_prompts_simple_generiated_images.csv", index=False)

In [ ]:
df['generated_artwork_name'].isnull().sum()

In [ ]:
for index, row in df.iterrows():
    if pd.isnull(row['generated_artwork_name']):
        print(index)

In [ ]:
df.head() 

In [ ]:
path_gen = "../data/artwork_gen_dalle"
path_orig = "../data/original_artworks_named"
files = glob.glob(path_gen + "/*")
len(files)

In [50]:
df = pd.read_csv("../data/artworks_data_with_prompts_simple_generiated_images.csv")

In [ ]:
df.isnull().sum()

In [52]:
df.to_csv("../data/artworks_data_with_prompts_simple_generated_images.csv", index=False)

In [ ]:
df.isnull().sum()

In [ ]:
path_gen = "../data/artwork_gen_dalle"
path_orig = "../data/original_artworks_named"
files = glob.glob(path_gen + "/*")
len(files)

In [ ]:
# iterate all rows in df 
ill_prompts = []
not_found = []
count = 0
for index, row in df.iterrows():
    
    print("==========" , count , "==========")
    
    if pd.isnull(row['generated_artwork_name']):
        print("couldn't generate")
        ill_prompts.append(index)
        count += 1
        continue

    img_name = row['generated_artwork_name']
    
    # extract the last part of the path after '/'
    img_name = img_name.split("/")[-1]
    # print("img_name", img_name)
    
    # search generated image in the path_gen
    found = False
    for file in files:
        if img_name in file:
            found = True
            break
    
    if not found:
        print("Image not found")
        not_found.append(index)
        count += 1
        continue
        
    count += 1

In [ ]:
len(ill_prompts)

In [ ]:
len(not_found)